Run this to get decision tree which we use as our classifier

In [27]:
%run "../DecisionTree/CS5350HW1.ipynb"

[1. 1. 1. ... 1. 1. 1.]
Starting Info:  0.5270569097708636
age 0.0007462669870993421
Starting Info:  0.5270569097708636
J [['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['yes', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['yes', 1.0], ['no', 1.0], ['yes', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['yes', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['yes', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['yes', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['no', 1.0], ['yes', 1.0], ['no', 1.0], ['no', 1.0],

# Adaboost Algorithm

## Main Algorithm

In [28]:
#Define D_t
#for t=1,2,..,T
    #find h_t whose weighted classification error is better than chance
    #Compute a_t = 1/2ln((1-e_t)/e_t)
    #Update D_t+1(i) = D_t(i)/Z_t*e^{-a_ty_ih_t(x_i)}
#H_final = sgn(sum(a_th_t(x)))

def Adaboost(D_t, iterations, s_train, full_attributes, attributes, purity_function):
    classifiers = []
    alphas = []
    for i in range(iterations):
        #print("Iteration:", i)
        root = createTree(s_train, attributes, full_attributes, purity_function, D_t, 1)
        print(root.label)
        classifiers.append(copy.deepcopy(root))
        error = findError(root, s_train, D_t, full_attributes)
        a_t = 0.5*np.log((1-error)/error)
        D_t_next = np.ones(len(s_train))
        for trainInd in range(len(s_train)):
            #travTree returns 1 if matched and 0 otherwise
            pred = travTree(root, s_train[trainInd], full_attributes)
            if(pred == 0):
                D_t_next[trainInd] = D_t[trainInd]*np.e**(a_t)
            else:
                D_t_next[trainInd] = D_t[trainInd]*np.e**(-a_t)
        #print(D_t_next)
        Z_t = np.sum(D_t_next)
        D_t_next = D_t_next/Z_t
        #print(D_t)
        #print(D_t_next)
        #print(error)
        #displayTree(root)
        D_t = D_t_next
        alphas.append(a_t)
    return classifiers, alphas, D_t
            
            

In [29]:
def findError(root, s, D_t, full_attributes):
  matched = 0
  for i in range(len(s)):
    #Note, (1-x) maps 0 -> 1 and 1 -> 0
    matched += D_t[i]*(1-travTree(root, s[i], full_attributes))
  return matched

## Finding accuracy

In [30]:
def findModelAccuracy(s_test, classifiers, alphas, full_attributes):
    matched = 0
    for test in s_test:
        h_f = computeHFinal(classifiers, alphas, test, full_attributes)
        #print(h_f, test[len(test)-1])
        if(h_f > 0 and convertStringToBinary(test[len(test)-1]) > 0):
            matched += 1
        elif(h_f < 0 and convertStringToBinary(test[len(test)-1]) < 0):
             matched += 1
    return matched/len(s_test)


In [31]:
def computeHFinal(classifiers, alphas, x_i, full_attributes):
    final = 0
    for i in range(len(classifiers)):
        #print(alphas[i])
        #print(travTreeWithoutLabel(classifiers[i], x_i, full_attributes))
        final += alphas[i]*travTreeWithoutLabel(classifiers[i], x_i, full_attributes)
    return final

In [32]:
def convertStringToBinary(y_i):
    if(y_i == "yes"):
        return 1
    else:
        return -1

Pretty much travTree, but now will return 1 for a yes label and -1 for a no label

In [33]:
def travTreeWithoutLabel(root, datapoint, full_attr):
  #Stopping Condition, root no children
  if(len(root.children) == 0):
    if(root.label == "yes"):
      #print("matched")
      return 1
    else:
      #print("no match")
      return -1
  #Otherwise, traverse down
  curr_attr = root.label
  do_numeric = False
  if(full_attr[curr_attr] == "numeric"):
    do_numeric = True
  #print("current attribute: ", curr_attr)
  atr_ind = list(full_attr.keys()).index(curr_attr)
  for child in root.children:
    if(do_numeric):
      median = statistics.median(x[atr_ind] for x in root.s)
      #print("numeric: ", median, " ", datapoint[atr_ind])
      if((datapoint[atr_ind] < median and child.parentVal=="lower than median") or (datapoint[atr_ind] >= median and child.parentVal=="higher than median")):
        return travTreeWithoutLabel(child, datapoint, full_attr)
    else:
      if(child.parentVal == datapoint[atr_ind]):
        #print("Going down on val: ", child.parentVal)
        return travTreeWithoutLabel(child, datapoint, full_attr)

# Testing

iterations = 1
s_train = s_banktraining
s_test = s_banktesting
full_attributes = full_attributes_bank
attributes = remaining_attributes_bank
purity_function = GiniInd

D_1 = np.ones(len(s_train))/len(s_train)

classifiers, alphas = Adaboost(D_t=D_1, iterations=iterations, s_train=s_train, full_attributes=full_attributes, attributes=attributes, purity_function=purity_function)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(alphas)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

T=500
classifierArr = []
alphaArr = []
D_t = np.ones(len(s_train))/len(s_train)
s_train = s_banktraining
s_test = s_banktesting
full_attributes = full_attributes_bank
attributes = remaining_attributes_bank
purity_function = GiniInd

for i in range(1, T):
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print(i)
    classifiers, alphas, weights = Adaboost(D_t=D_t, iterations=1, s_train=s_train, full_attributes=full_attributes, attributes=attributes, purity_function=purity_function)
    classifierArr.append(classifiers[0])
    alphaArr.append(alphas[0])
    D_t = weights
    print("train:", findModelAccuracy(s_test=s_train, classifiers=classifierArr, alphas=alphaArr, full_attributes=full_attributes))
    print("test:",findModelAccuracy(s_test=s_test, classifiers=classifierArr, alphas=alphaArr, full_attributes=full_attributes))